In [1]:
# !pip install --upgrade bitsandbytes 

In [2]:
# !pip install --upgrade git+https://github.com/huggingface/peft.git 
# !pip install --upgrade git+https://github.com/huggingface/transformers.git
# !pip install --upgrade git+https://github.com/huggingface/accelerate.git
# !pip install --upgrade datasets
# !pip install --upgrade loralib


In [3]:
!module load cudnn
!module load cuda

Loading module for cudnn
cudnn is now loaded
Loading module for CUDA 9.2.148.1
CUDA 9.2.148.1 is now loaded
Loading module for CUDA 11.2.2
CUDA 11.2.2 is now loaded


In [4]:
import torch

torch.cuda.is_available()

True

In [5]:
!nvidia-smi

Wed Jul 12 11:57:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:B3:00.0 Off |                    0 |
| N/A   30C    P0    55W / 300W |   2701MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import os
print(os.environ.get('PYTHONPATH'))

None


In [8]:
import os
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import bitsandbytes as bnb
import triton

print(triton.__path__)

model_name = "/home/toibazd/Data/Text2SQL/content/t5-small-finetuned-wikisql"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map = 'auto')
tokenizer = AutoTokenizer.from_pretrained(model_name)

False
'CUDASetup' object has no attribute 'cuda'


/home/toibazd/miniconda3/lib/python3.9/site-packages/bitsandbytes-0.40.0.post4-py3.9.egg/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/toibazd/miniconda3/lib/python3.9/site-packages/bitsandbytes-0.40.0.post4-py3.9.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: /home/toibazd/miniconda3 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/toibazd/miniconda3/lib/python3.9/site-packages/bitsandbytes-0.40.0.post4-py3.9.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('CentOS/7.9.2009')}
  warn(msg)


['/home/toibazd/miniconda3/lib/python3.9/site-packages/triton']
[2023-07-12 11:57:47,496] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-07-12 11:57:49.204870: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 11:57:49.373989: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-12 11:57:49.419668: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-12 11:57:51.226184: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [9]:
print(model)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [10]:
for param in model.parameters():
    param.requires_grad = False
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)
        
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

In [11]:
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [12]:
def get_trainable_params(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")
get_trainable_params(model)

trainable params: 0 || all params: 60506624 || trainable%: 0.0


In [13]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType


config = LoraConfig(
    r = 8,
    lora_alpha = 16,
    lora_dropout=0.05,
    task_type= TaskType.SEQ_2_SEQ_LM,
    bias = "none"
)

model = prepare_model_for_int8_training(model)
model = get_peft_model(model, config)
get_trainable_params(model)

trainable params: 294912 || all params: 60801536 || trainable%: 0.4850403779272945


/home/toibazd/miniconda3/lib/python3.9/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [15]:
df1 = pd.read_csv('Training_set_IMDB/training_set_sample_no_target.csv')
df1 = df1.dropna()


shuffled_df = df1.sample(frac=1, random_state=42)  # Set random_state for reproducibility

train_df, eval_df = train_test_split(shuffled_df, test_size=0.1, random_state=42)

# 'train_df' will contain 80% of the shuffled data for training
# 'eval_df' will contain 20% of the shuffled data for evaluation


text = []
sql = []

for index, row in train_df.iterrows():
    
    text_input = "Translate to SQL: " + row['Text'],
    sql_input = row['SQL']
    text.append(text_input)
    sql.append(sql_input)
    
inputs = {"inputs": text,
        "target": sql}

train_dataset = Dataset.from_dict(inputs)

In [16]:
text_2 = []
sql_2 = []

for index, row in eval_df.iterrows():
    
    text_input = "Translate to SQL: " + row['Text'],
    sql_input = row['SQL']
    text_2.append(text_input)
    sql_2.append(sql_input)
    
inputs_2 = {"inputs": text_2,
        "target": sql_2}
eval_dataset = Dataset.from_dict(inputs_2)

In [17]:
eval_dataset['inputs'][0]

['Translate to SQL: what is actor Jack (VI) Lewis first role?']

In [18]:
def map_to_lenght(x):
    x["input_len"] = len(tokenizer(x["inputs"]).input_ids)
    x["input_longer_256"] = int(x["input_len"]>256)
    x["input_longer_128"] = int(x["input_len"]>128)
    x["input_longet_64"] = int(x["input_len"]>64)
    x["output_len"] = len(tokenizer(x["target"]).input_ids)
    x["output_longet_256"] = int(x["output_len"]>256)
    x["output_longet_128"] = int(x["output_len"]>128)
    x["output_longet_64"] = int(x["output_len"]>64)
    return x

sample_size = 10000
data_stats = train_dataset.select(range(sample_size)).map(map_to_lenght, num_proc=4)

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [19]:
def compute_and_print(x):
    if len(x["input_len"])==sample_size:
        print(
            f"Input mean: {sum(x['input_len'])/sample_size} \n % of input len > 256: {sum(x['input_longer_256'])/sample_size}, \n % of input len > 128: {sum(x['input_longer_128'])/sample_size}, \n % of input len > 64: {sum(x['input_longet_64'])/sample_size}, \n Ouput mean: {sum(x['output_len'])/sample_size},\n% of output len > 256: {sum(x['output_longet_256'])/sample_size}, \n% of output len > 128: {sum(x['output_longet_128'])/sample_size}, \n% of output len > 64: {sum(x['output_longet_64'])/sample_size}")

output = data_stats.map(compute_and_print, batched=True, batch_size=-1)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Input mean: 1.0 
 % of input len > 256: 0.0, 
 % of input len > 128: 0.0, 
 % of input len > 64: 0.0, 
 Ouput mean: 106.4648,
% of output len > 256: 0.0, 
% of output len > 128: 0.2934, 
% of output len > 64: 0.7241


In [20]:
train_dataset["target"][-500:-1]

['SELECT COUNT(distinct r.role) AS role_count FROM roles r JOIN movies m ON r.movie_id = m.id WHERE m.name = \'"Affre Semmeling, Die"\'',
 "SELECT COUNT(*) AS movie_count FROM movies m JOIN movies_genres mg ON m.id = mg.movie_id WHERE m.year = 1939 AND mg.genre = 'Animation'",
 "SELECT r.role AS recent_role FROM actors a INNER JOIN roles r ON a.id = r.actor_id INNER JOIN movies m ON r.movie_id = m.id WHERE a.first_name = 'Gary' AND a.last_name = 'Beisel' ORDER BY m.year DESC LIMIT 1",
 "SELECT d.id AS director_id, d.first_name, d.last_name, COUNT(m.id) AS movie_count FROM directors d JOIN movies_directors md ON d.id = md.director_id JOIN movies m ON md.movie_id = m.id WHERE d.first_name = 'Hans-Jürgen' AND d.last_name = 'Tögel' AND m.year = 1996 GROUP BY d.id, d.first_name, d.last_name",
 "SELECT genre FROM directors t1 JOIN directors_genres t2 ON t1.id = t2.director_id WHERE t1.first_name = 'Wally (II)' AND t1.last_name = 'White'",
 "SELECT m.* FROM movies m JOIN movies_directors md O

In [21]:
def convert_to_features(example_batch, padding = "max_length"):
    inputs = tokenizer.batch_encode_plus(example_batch["inputs"],is_split_into_words=True, max_length=64, padding="max_length", truncation=True)
    targets = tokenizer.batch_encode_plus(example_batch["target"], max_length=256, padding="max_length",truncation = True)
    if padding == "max_length":
        targets["inputs_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in target] for target in targets["input_ids"]
        ]
        
    encodings = {
        "input_ids":inputs["input_ids"],
        "attention_mask":inputs["attention_mask"],
        "labels":targets["input_ids"],
        "decoder_attention_mask":targets["attention_mask"]
    }
    return encodings

train_data = train_dataset.map(convert_to_features, batched=True, remove_columns=train_dataset.column_names)
test_data = eval_dataset.map(convert_to_features, batched=True, remove_columns=eval_dataset.column_names)

Map:   0%|          | 0/38756 [00:00<?, ? examples/s]

Map:   0%|          | 0/4307 [00:00<?, ? examples/s]

In [22]:
columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

train_data.set_format(type='torch', columns=columns)
test_data.set_format(type='torch', columns=columns)

In [23]:
len(test_data["input_ids"])

4307

In [24]:
def sequence_check(list_of_seq):
    len_seq = len(list_of_seq[0])
    for seq in list_of_seq:
        if len(seq)!=len_seq:
            print(len(seq))
            return False
        
    return True

In [25]:
sequence_check(train_data['input_ids'])

True

In [26]:
len(train_data["input_ids"][12330])


64

In [27]:
from transformers import Seq2SeqTrainingArguments
output_dir = '/home/toibazd/Data/Text2SQL/LoRA_adapted_T5'
args = Seq2SeqTrainingArguments(
        auto_find_batch_size=True,
        gradient_accumulation_steps = 4,
        learning_rate=6e-4, 
        warmup_steps=100,  
        save_strategy="no",
        load_best_model_at_end=True,
        num_train_epochs=10,
        eval_accumulation_steps=10,
        fp16=True,
        logging_steps=100, 
        output_dir=output_dir)

In [28]:
! pip install -q rouge_score evaluate


In [29]:
from evaluate import load
rouge = load("rouge")

In [30]:
def compute_metrics(pred):
    label_ids = pred.label_ids
    pred_ids = pred.predictions
    
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids== -101] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
    print_gpu_utilization()
    return {
        "rouge2 precision": round(rouge_output.precision, 4),
        "rouge2 recall": round(rouge_output.recall, 4),
        "rouge2 F1 score": round(rouge_output.fmeasure, 4)
    }

In [31]:
from transformers import Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq

label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                      model=model,
                                      label_pad_token_id=label_pad_token_id)

trainer = Seq2SeqTrainer(
    model = model,
    train_dataset = train_data,
    args = args,
    data_collator=data_collator
)

model.config.use_cache = False

In [32]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

In [33]:
trainer.train()

/home/toibazd/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,3.017800
200,0.841800
300,0.414100
400,0.250300
500,0.171200
600,0.125100
700,0.096200
800,0.077100
900,0.068600
1000,0.059100


TrainOutput(global_step=12110, training_loss=0.06082265077667921, metrics={'train_runtime': 8306.3351, 'train_samples_per_second': 46.658, 'train_steps_per_second': 1.458, 'total_flos': 6599228918857728.0, 'train_loss': 0.06082265077667921, 'epoch': 10.0})

In [34]:
trainer.save_model()
tokenizer.save_pretrained(output_dir)


('/home/toibazd/Data/Text2SQL/LoRA_adapted_T5/tokenizer_config.json',
 '/home/toibazd/Data/Text2SQL/LoRA_adapted_T5/special_tokens_map.json',
 '/home/toibazd/Data/Text2SQL/LoRA_adapted_T5/tokenizer.json')